In [30]:
# maybe just remove 'Tag entirely'
# consider just looking at 1 type of tournament 

In [1]:
import pandas as pd
import sqlite3
import numpy as np 
from datetime import datetime, date, timedelta 

pd.set_option("display.max_columns", 100)
conn = sqlite3.connect("drivehud.db")


df = pd.read_sql_query("select * from Tournaments;", conn) # pretty sure these all connect to form a more full picture


In [2]:
def just_me(input_df):
    df = input_df.copy()
    mask = df.PlayerId == 1
    df_me = df[mask]
    return df_me

def remove_extra_columns(input_df):
    # tournament size most likely riddled with errors, but may be useful for <= 6 
    # consider looking at bounty but as of now it seems as though it is useless 
    df = input_df.copy()
    df.drop(['PokerSiteId', 'TournamentId', 'Currency', 'GameType', 'PlayerId', 'FileLastModifiedTime', 'ImportType', 'Bounty', 'Filename', 'PlayerFinished'], axis = 1, inplace = True)
    return df 

def net_winnings(input_df):
    df = input_df.copy()
    df['NetWinnings'] = df.Winnings - df.BuyIn - df.Rake
    return df 

def to_datetime(input_df):
    df = input_df.copy()
    df['FirstHandTimestamp'] = pd.to_datetime(df['FirstHandTimestamp'])
    df['LastHandTimestamp'] = pd.to_datetime(df['LastHandTimestamp'])
    return df

def create_roi(input_df):
    df = input_df.copy()
    #df['ROI'] = 0
    df['ROI'] = np.where(df['BuyIn']!= 0 ,(100 * df['NetWinnings'] / (df['BuyIn'] + df['Rake'])), 'ignore')
    #df['ROI'] = np.where(df['ROI']!= 'ignore', )
    return df

def total_buyin(input_df):
    df = input_df.copy()
    df['total_buyin'] = df.BuyIn + df.Rake
    return df

def time_started(row):
    result = row.time().strftime('%H:%M')
    hour = int(result[:2])
    time = int(result[-2:]) / 60
    result = hour + time 
    return result

def time_finished(row):
    result = row.time().strftime('%H:%M')
    hour = int(result[:2])
    time = int(result[-2:]) / 60
    result = hour + time 
    return result

def date_of_game(input_df):
    df = input_df.copy()
    df['date'] = df.FirstHandTimestamp.dt.date 
    return df

def day_progression(input_df):
    df = input_df.copy()
    to_sub = df.date.min()
    
    df['days_since_start'] = (df.date - to_sub)
    df['days_since_start'] = df.days_since_start.dt.days
    return df

def above_combined(input_df):
    df = input_df.copy()
    df = just_me(df)
    df = remove_extra_columns(df)
    df = net_winnings(df)
    df = to_datetime(df)
    df = create_roi(df)
    df = total_buyin(df)
    df['time_started'] = df.FirstHandTimestamp.apply(lambda row: time_started(row))
    df['time_ended'] = df.LastHandTimestamp.apply(lambda row: time_finished(row))
    df = date_of_game(df)
    df = day_progression(df)
    return df 

In [3]:
df = above_combined(df)


In [208]:
def weekly_df(input_df):
    df = input_df.copy()
    result = pd.DataFrame()
    consider = np.array([0,1,2,3,4,5,6])
    idx = 0
    
    while consider[0] <= df.days_since_start.max():
        temp = pd.DataFrame()
        for num in consider:
            mask = df.days_since_start == num
            temp = temp.append(df[mask])
        if len(temp) > 0:
            mask = temp.Tag == 'MTT'
            mtt = temp[mask]
            percent_mtt = len(mtt) / len(temp)
            
            single_row = pd.DataFrame({'percent_mtt':  percent_mtt, 
                                      'SpeedType': temp.SpeedType.mean(),
                                      'TournamentSize': temp.TournamentSize.mean(),
                                      'TableSize' : temp.TableSize.mean(),
                                      'StartingStacks': temp.StartingStacks.mean(),
                                      'NetWinnings'  : temp.NetWinnings.sum(),
                                       'Winnings' : temp.Winnings.sum(),
                                      'total_buyin' : temp.total_buyin.sum(), 
                                      'time_started' : temp.time_started.mean(),
                                      'time_ended' : temp.time_ended.mean(), 
                                      'days_since_start' : temp.days_since_start.mean(),
                                      'tournaments_played' : len(temp)},
                                         index = [idx])
            result = result.append(single_row)
            idx += 1
        consider += 7 
        result['ROI'] = 100 * result['NetWinnings'] / result['total_buyin'] 
    return result 

In [209]:
weekly = weekly_df(df)

In [210]:
weekly

,percent_mtt,SpeedType,TournamentSize,TableSize,StartingStacks,NetWinnings,Winnings,total_buyin,time_started,time_ended,days_since_start,tournaments_played,ROI
0,1.000000,1.428571,24.285714,9.000000,5392.857143,-580,5360,5940,9.002381,10.447619,2.357143,14,-9.764310
1,1.000000,1.272727,23.727273,9.000000,6909.090909,-4991,1719,6710,7.719697,8.939394,11.272727,11,-74.381520
2,1.000000,1.142857,65.571429,9.000000,5571.428571,1028,3558,2530,6.826190,8.757143,16.571429,7,40.632411
3,1.000000,1.000000,27.666667,9.000000,6500.000000,-1980,0,1980,2.508333,3.930556,22.833333,6,-100.000000
4,0.666667,1.666667,14.000000,7.000000,4500.000000,-640,0,640,9.616667,10.050000,31.333333,3,-100.000000
5,0.250000,1.000000,10.000000,4.500000,1125.000000,-710,0,710,5.120833,5.445833,52.500000,4,-100.000000
6,1.000000,1.000000,23.333333,9.000000,2500.000000,-420,0,420,3.250000,4.255556,58.666667,3,-100.000000
7,1.000000,1.000000,29.333333,9.000000,3000.000000,-97,233,330,3.488889,4.833333,66.000000,3,-29.393939
8,1.000000,1.000000,22.500000,9.000000,5750.000000,-1100,0,1100,1.925000,3.108333,74.000000,2,-100.000000
9,1.000000,1.000000,25.000000,9.000000,3000.000000,-110,0,110,3.450000,4.083333,86.000000,1,-100.000000


array([ 7,  8,  9, 10, 11, 12, 13])

,TournamentNumber,Tag,BuyIn,Rake,SpeedType,TablesQty,TournamentSize,TableSize,StartingStacks,FirstHandTimestamp,LastHandTimestamp,PlayerEndPosition,Winnings,Rebuy,NetWinnings,ROI,total_buyin,time_started,time_ended,date,days_since_start
0,459742,MTT,100,10,1,2,18,9,1500,2019-10-24 03:08:08,2019-10-24 04:45:45,1,972,0,862,783.6363636363636,110,3.133333,4.750000,2019-10-24,0
18,590440,MTT,300,30,3,2,18,9,10000,2019-10-24 05:38:11,2019-10-24 06:09:01,50,0,0,-330,-100.0,330,5.633333,6.150000,2019-10-24,0
36,4195484,MTT,500,50,1,1,9,9,1500,2019-10-24 15:32:24,2019-10-24 16:06:44,21,0,0,-550,-100.0,550,15.533333,16.100000,2019-10-24,0
45,1377658,MTT,100,10,1,2,13,9,1500,2019-10-24 16:21:48,2019-10-24 17:17:13,13,0,0,-110,-100.0,110,16.350000,17.283333,2019-10-24,0
58,2753680,MTT,100,10,1,2,12,9,1500,2019-10-24 20:25:41,2019-10-24 21:16:48,13,0,0,-110,-100.0,110,20.416667,21.266667,2019-10-24,0
70,525994,MTT,500,50,1,7,58,9,10000,2019-10-25 00:32:50,2019-10-25 04:41:29,22,1312,0,762,138.54545454545453,550,0.533333,4.683333,2019-10-25,1
128,132212,MTT,700,70,2,3,27,9,10000,2019-10-26 02:32:38,2019-10-26 03:37:23,63,200,0,-570,-74.02597402597402,770,2.533333,3.616667,2019-10-26,2
155,459824,MTT,100,10,1,2,16,9,1500,2019-10-26 15:32:18,2019-10-26 16:10:37,9,0,0,-110,-100.0,110,15.533333,16.166667,2019-10-26,2
171,525150,MTT,700,70,2,5,42,9,10000,2019-10-28 02:32:52,2019-10-28 04:46:15,42,1195,0,425,55.1948051948052,770,2.533333,4.766667,2019-10-28,4
213,459164,MTT,100,10,1,2,15,9,1500,2019-10-28 15:23:21,2019-10-28 16:07:51,7,0,0,-110,-100.0,110,15.383333,16.116667,2019-10-28,4


,percent_mtt,SpeedType,TournamentSize,TableSize,StartingStacks,NetWinnings,total_buyin,time_started,time_ended,days_since_start
0,1.0,1.428571,24.285714,9.0,5392.857143,-580,5940,9.002381,10.447619,2.357143
